# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

from razdel import tokenize

In [4]:
data = pd.read_csv('labeled.csv')

In [3]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

In [4]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

In [6]:
y = train.toxic.values
y_test = test.toxic.values

In [7]:
razdel_tokenizer = lambda doc: [t.text for t in list(tokenize(doc))]
razdel_vectorizer = CountVectorizer(tokenizer=razdel_tokenizer)
X2 = razdel_vectorizer.fit_transform(train.comment)
X2_test = razdel_vectorizer.transform(test.comment) 

In [13]:
dt = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
dt.fit(X, y)
pred = dt.predict(X_test)

In [14]:
dt2 = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
dt2.fit(X2, y)
pred2 = dt2.predict(X2_test)

In [15]:
print('default classifier:')
print(classification_report(y_test, pred))
print('\n\nrazdel classifier:')
print(classification_report(y_test, pred2))

default classifier:
              precision    recall  f1-score   support

         0.0       0.80      0.58      0.67       951
         1.0       0.47      0.72      0.57       491

    accuracy                           0.63      1442
   macro avg       0.63      0.65      0.62      1442
weighted avg       0.69      0.63      0.64      1442



razdel classifier:
              precision    recall  f1-score   support

         0.0       0.77      0.72      0.74       951
         1.0       0.52      0.58      0.55       491

    accuracy                           0.67      1442
   macro avg       0.64      0.65      0.65      1442
weighted avg       0.68      0.67      0.68      1442



Раздел работает лучше, чем дефолтный токенизатор, но разница незначительная.

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [16]:
freqs = [[1, 1, 1, 0, 0, 0], 
         [1, 1, 1, 0, 0, 0], 
         [3, 0, 1, 1, 0, 0], 
         [1, 0, 0, 1, 1, 0], 
         [0, 0, 0, 0, 0, 1]]

In [17]:
table = pd.DataFrame(freqs, columns = ['я', 'ты', 'и', 'только', 'не', 'он'], index = ['я и ты','ты и я','я, я и только я', 'только не я', 'он'])
table

,я,ты,и,только,не,он
я и ты,1,1,1,0,0,0
ты и я,1,1,1,0,0,0
"я, я и только я",3,0,1,1,0,0
только не я,1,0,0,1,1,0
он,0,0,0,0,0,1


In [18]:
tfidf = table.apply(lambda x: x/ x.sum(), axis = 1)
tfidf

,я,ты,и,только,не,он
я и ты,0.333333,0.333333,0.333333,0.000000,0.000000,0.0
ты и я,0.333333,0.333333,0.333333,0.000000,0.000000,0.0
"я, я и только я",0.600000,0.000000,0.200000,0.200000,0.000000,0.0
только не я,0.333333,0.000000,0.000000,0.333333,0.333333,0.0
он,0.000000,0.000000,0.000000,0.000000,0.000000,1.0


In [30]:
frame = {}
for col in table.columns:
    N = len(table)
    df = len(table[col][table[col]!=0])
    series = tfidf[col].apply(lambda x: x*np.log10(N/df))
    frame[col] = series
result = pd.DataFrame(frame)
result

,я,ты,и,только,не,он
я и ты,0.032303,0.132647,0.07395,0.000000,0.00000,0.00000
ты и я,0.032303,0.132647,0.07395,0.000000,0.00000,0.00000
"я, я и только я",0.058146,0.000000,0.04437,0.079588,0.00000,0.00000
только не я,0.032303,0.000000,0.00000,0.132647,0.23299,0.00000
он,0.000000,0.000000,0.00000,0.000000,0.00000,0.69897


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [5]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [6]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1), lowercase=True, stop_words = None, encoding="utf-8")
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

In [7]:
y = train.toxic.values
y_test = test.toxic.values

In [8]:
clf = LogisticRegression(C=0.1, class_weight='balanced', max_iter = 300)

In [9]:
clf.fit(X, y)
preds_log = clf.predict(X_test)

In [10]:
print(classification_report(y_test, preds_log))

              precision    recall  f1-score   support

         0.0       0.92      0.84      0.88       965
         1.0       0.73      0.84      0.78       477

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.85      0.84      0.85      1442



In [11]:
#второй токенизатор для второго классификатора
vectorizer = TfidfVectorizer(analyzer='word', lowercase=True, min_df=2, max_df=0.3, max_features = 10000)
X2 = vectorizer.fit_transform(train.comment)
X2_test = vectorizer.transform(test.comment)

In [12]:
clf_NB = MultinomialNB(alpha = 0.5, fit_prior = True)
clf_NB.fit(X2, y)
preds_NB = clf_NB.predict(X2_test)

print(classification_report(y_test, preds_NB))

              precision    recall  f1-score   support

         0.0       0.84      0.97      0.90       965
         1.0       0.91      0.64      0.75       477

    accuracy                           0.86      1442
   macro avg       0.88      0.80      0.82      1442
weighted avg       0.87      0.86      0.85      1442



In [17]:
probas = clf.predict_proba(X_test)
probas_NB = clf_NB.predict_proba(X2_test)

In [23]:
def find_toxic(probas):
    probas = [p[1] for p in probas]
    test['probas'] = probas
    res_df = test.sort_values(by = 'probas', ascending = False)[:10]
    res_df = res_df.reset_index(drop=True)
    for i in range(10):
        print('toxic:', res_df.loc[i].toxic)
        print('comment: ', res_df.loc[i].comment)
    return res_df

In [25]:
print('CountVectorizer and LogisticRegression: ')
clf1 = find_toxic(probas)
print('TfidfVectorizer and MultinomialNB: ')
clf2 = find_toxic(probas_NB)

CountVectorizer and LogisticRegression: 
toxic: 1.0
comment:  Так ты не по теме гомофорса давай, а в общем. Нахуя ты машешь банхамером? Это не лично твоя доска - не нравится, как общаются некоторые аноны - пили свой сайт и там наводи собственные порядки. Сейчас все выглядит так, что ты встав с утра не с той ноги, начинаешь раздавать баны, увидев там некий намек на срач и пытаясь искусственно зарегулировать доску под какое-то свое виденье того, как доска должна выглядить. Системность следования правилам раздела, отсутствие избирательного отношения или фаворитизма по отношению к тому или иному форсу, - залог стабильного постинга на честной доске. Что ты несешь, шизофреник задроченный? Честная доска? Ты таблеток принял? Кто тебе вообще дал ограничивать право на форс других людей, к тому же относительно фентези сука. Это игра, здесь все выдуманное, але. И вообще, что ты имеешь против гомиков, говна кусок? Ты уже не стесняешься и решил открыто ссать всем обитателям tes в лицо? надоели и ста

<ipython-input-23-f925beb516d6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['probas'] = probas


In [27]:
clf1.merge(clf2, how = 'inner', on ='comment')

,index_x,comment,toxic_x,probas_x,index_y,toxic_y,probas_y
0,439,"Так ты не по теме гомофорса давай, а в общем. ...",1.0,1.000000,439,1.0,0.985885
1,241,хорошо тут все ясно а теперь поясни мне за то ...,1.0,0.999938,241,1.0,0.982721
2,380,"В очередной раз убеждаюсь, что двачеры редкост...",1.0,0.998808,380,1.0,0.987138
3,96,Э че бля а? Ты че нах бл иди сюда епты.. Ты за...,1.0,0.997980,96,1.0,0.989602


Тексты правда очень таксичные....совпали только 4 комментария. В первом классификаторе тексте подлиннее оказались. Хотя совпали 2 очень длинных, 1 средний и 1 короткий, так что, скорее всего, никакого интересного наблюдения тут нет.

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [41]:
import nltk

In [42]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gryaz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [43]:
from nltk.corpus import stopwords

In [44]:
stopwords

<WordListCorpusReader in 'C:\\Users\\gryaz\\AppData\\Roaming\\nltk_data\\corpora\\stopwords'>

In [45]:
train, test = train_test_split(data, test_size=0.1, random_state = 100)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
y = train.toxic.values
y_test = test.toxic.values
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1), lowercase=True, stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X, y)
importance = clf.coef_
words ={value:key for key, value in vectorizer.vocabulary_.items()}
idx = importance.argsort()
for i in idx[0][:-6:-1]:
    print(words[i])

хохлы
хохлов
тебе
дебил
нахуй


In [46]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1), lowercase=True, stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
clf.fit(X, y)
importance = clf.feature_importances_
words ={value:key for key, value in vectorizer.vocabulary_.items()}
idx = importance.argsort()
for i in idx[:-6:-1]:
    print(words[i])

тебе
хохлы
очень
нахуй
хохлов


In [48]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1), lowercase=True, stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = MultinomialNB(alpha = 0.5, fit_prior = True)
clf.fit(X, y)
importance = clf.coef_
words ={value:key for key, value in vectorizer.vocabulary_.items()}
idx = importance.argsort()
for i in idx[0][:-6:-1]:
    print(words[i])

это
просто
тебе
всё
вообще


C:\Users\gryaz\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute coef_ was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


In [49]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1), lowercase=True, stop_words=stopwords.words('russian'))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
clf = RandomForestClassifier(n_estimators=100, max_depth=20)
clf.fit(X, y)
importance = clf.feature_importances_
words ={value:key for key, value in vectorizer.vocabulary_.items()}
idx = importance.argsort()
for i in idx[:-6:-1]:
    print(words[i])

тебе
хохлы
блядь
быдло
тред


Практически во всех топах, кроме Наивного Байеса, присутствуют словоформы лексему хохлы, хорошо бы предварительно тексты лемматизировать. И можно еще убрать дополнительно слова "это", "всё", "просто" и т.д., которые как раз вылезли в наивном байесе почему-то.